In [ ]:
import json
import pandas as pd
import re
from google.cloud import bigquery
from google.oauth2 import service_account
import os
from datetime import datetime
from datetime import date

In [4]:
with open("koenigsegg.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)
    f.close()

In [5]:
x = []
for i in data: # Loop through every page
    for j in i: # Loop through every individual car listing on a specific page
        x.append(j)

x = pd.DataFrame(x)


In [6]:
y = x.copy()
y.replace(to_replace="", value=None, inplace=True)
y["leistung"] = y["leistung"].apply(lambda x: int(re.findall(pattern="(?<=\()\d+", string=x)[0]) if x is not None else x)
y["preis"] = y["preis"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
y["kilometer"] = y["kilometer"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
y["fahrzeughalter"] = y["fahrzeughalter"].apply(lambda x: int(x) if x is not None else x)
y["standort"] = y["standort"].apply(lambda x: re.findall(pattern="[A-za-z]+(?=-)", string=x)[0] if x is not None else x)

In [7]:
y["crawled_timestamp"] = datetime.now()

In [12]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   marke                520 non-null    object        
 1   modell               520 non-null    object        
 2   variante             0 non-null      object        
 3   titel                520 non-null    object        
 4   form                 520 non-null    object        
 5   fahrzeugzustand      325 non-null    object        
 6   leistung             471 non-null    float64       
 7   getriebe             511 non-null    object        
 8   farbe                10 non-null     object        
 9   preis                520 non-null    int64         
 10  kilometer            514 non-null    float64       
 11  erstzulassung        482 non-null    object        
 12  fahrzeughalter       247 non-null    float64       
 13  standort             520 non-null  

In [74]:
y[y["farbe"].isnull()].loc[0, "url_to_crawl"]

'https://suchen.mobile.de/fahrzeuge/details.html?id=296584429&damageUnrepaired=ALSO_DAMAGE_UNREPAIRED&isSearchRequest=true&makeModelVariant1.makeId=13900&pageNumber=1&scopeId=C&fnai=prev&searchId=bda3cb2f-5499-bbae-6d80-24ecea55a156&ref=srp'

In [13]:
# First, set the credentials
key_path = os.getcwd() + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("marke", "STRING"),
        bigquery.SchemaField("modell", "STRING"),
        bigquery.SchemaField("variante", "STRING"),
        bigquery.SchemaField("titel", "STRING"),
        bigquery.SchemaField("form", "STRING"),
        bigquery.SchemaField("fahrzeugzustand", "STRING"),
        bigquery.SchemaField("leistung", "FLOAT64"),
        bigquery.SchemaField("getriebe", "STRING"),
        bigquery.SchemaField("farbe", "STRING"),
        bigquery.SchemaField("preis", "INT64"),
        bigquery.SchemaField("kilometer", "FLOAT64"),
        bigquery.SchemaField("erstzulassung", "STRING"),
        bigquery.SchemaField("fahrzeughalter", "FLOAT64"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("fahrzeugbescheibung", "STRING"),
        bigquery.SchemaField("url_to_crawl", "STRING"),
        bigquery.SchemaField("page_rank", "INT64"),
        bigquery.SchemaField("total_num_pages", "INT64"),
        bigquery.SchemaField("crawled_timestamp", "TIMESTAMP"),
    ]
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

# Upload the table
job = client.load_table_from_dataframe(
    dataframe=y,
    destination="web-scraping-371310.crawled_datasets.lukas_mobile_de",
    job_config=job_config
).result()